<a href="https://colab.research.google.com/github/jiyun1006/deeplearning-pytorch/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**2. Word2Vec**
1. 주어진 단어들을 word2vec 모델에 들어갈 수 있는 형태로 만듭니다.
2. CBOW, Skip-gram 모델을 각각 구현합니다.
3. 모델을 실제로 학습해보고 결과를 확인합니다.

### **필요 패키지 import**

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 12.8MB/s 
     |████████████████████████████████| 460kB 50.2MB/s 
     |████████████████████████████████| 92kB 11.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [10]:
from tqdm import tqdm
from konlpy.tag import Okt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

import torch
import copy
import numpy as np

### **데이터 전처리**



데이터를 확인하고 Word2Vec 형식에 맞게 전처리합니다.  
학습 데이터는 1번 실습과 동일하고, 테스트를 위한 단어를 아래와 같이 가정해봅시다.

In [9]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."       
]

test_words = ["음식", "맛", "서비스", "위생", "가격"]

Tokenization과 vocab을 만드는 과정은 이전 실습과 유사합니다.

In [6]:
tokenizer = Okt() #Openkoreatext

In [7]:
def make_tokenized(data):
  tokenized = []
  for sent in tqdm(data):
    tokens = tokenizer.morphs(sent, stem=True)  #morphs 형태소 추출
    tokenized.append(tokens)

  return tokenized

In [11]:
train_tokenized = make_tokenized(train_data)
print(train_tokenized)

100%|██████████| 10/10 [00:05<00:00,  1.82it/s]

[['정말', '맛있다', '.', '추천', '하다', '.'], ['기대하다', '것', '보단', '별로', '이다', '.'], ['다', '좋다', '가격', '이', '너무', '비싸다', '다시', '가다', '싶다', '생각', '이', '안', '드네', '요', '.'], ['완전', '최고', '이다', '!', '재', '방문', '의사', '있다', '.'], ['음식', '도', '서비스', '도', '다', '만족스럽다', '.'], ['위생', '상태', '가', '좀', '별로', '이다', '.', '좀', '더', '개선', '되다', '기르다', '바라다', '.'], ['맛', '도', '좋다', '직원', '분들', '서비스', '도', '너무', '친절하다', '.'], ['기념일', '에', '방문', '하다', '음식', '도', '분위기', '도', '서비스', '도', '다', '좋다', '.'], ['전반', '적', '으로', '음식', '이', '너무', '짜다', '.', '저', '는', '별로', '이다', '.'], ['위생', '에', '조금', '더', '신경', '써다', '좋다', '.', '조금', '불쾌하다', '.']]


In [12]:
word_count = defaultdict(int)

for tokens in tqdm(train_tokenized):
  for token in tokens:
    word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 9670.98it/s]


In [13]:
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print(list(word_count))

[('.', 14), ('도', 7), ('이다', 4), ('좋다', 4), ('별로', 3), ('다', 3), ('이', 3), ('너무', 3), ('음식', 3), ('서비스', 3), ('하다', 2), ('방문', 2), ('위생', 2), ('좀', 2), ('더', 2), ('에', 2), ('조금', 2), ('정말', 1), ('맛있다', 1), ('추천', 1), ('기대하다', 1), ('것', 1), ('보단', 1), ('가격', 1), ('비싸다', 1), ('다시', 1), ('가다', 1), ('싶다', 1), ('생각', 1), ('안', 1), ('드네', 1), ('요', 1), ('완전', 1), ('최고', 1), ('!', 1), ('재', 1), ('의사', 1), ('있다', 1), ('만족스럽다', 1), ('상태', 1), ('가', 1), ('개선', 1), ('되다', 1), ('기르다', 1), ('바라다', 1), ('맛', 1), ('직원', 1), ('분들', 1), ('친절하다', 1), ('기념일', 1), ('분위기', 1), ('전반', 1), ('적', 1), ('으로', 1), ('짜다', 1), ('저', 1), ('는', 1), ('신경', 1), ('써다', 1), ('불쾌하다', 1)]


In [14]:
w2i = {}
for pair in tqdm(word_count):
  if pair[0] not in w2i:
    w2i[pair[0]] = len(w2i)

100%|██████████| 60/60 [00:00<00:00, 102051.19it/s]


In [ ]:
print(train_tokenized)
print(w2i)

[['정말', '맛있다', '.', '추천', '하다', '.'], ['기대하다', '것', '보단', '별로', '이다', '.'], ['다', '좋다', '가격', '이', '너무', '비싸다', '다시', '가다', '싶다', '생각', '이', '안', '드네', '요', '.'], ['완전', '최고', '이다', '!', '재', '방문', '의사', '있다', '.'], ['음식', '도', '서비스', '도', '다', '만족스럽다', '.'], ['위생', '상태', '가', '좀', '별로', '이다', '.', '좀', '더', '개선', '되다', '기르다', '바라다', '.'], ['맛', '도', '좋다', '직원', '분들', '서비스', '도', '너무', '친절하다', '.'], ['기념일', '에', '방문', '하다', '음식', '도', '분위기', '도', '서비스', '도', '다', '좋다', '.'], ['전반', '적', '으로', '음식', '이', '너무', '짜다', '.', '저', '는', '별로', '이다', '.'], ['위생', '에', '조금', '더', '신경', '써다', '좋다', '.', '조금', '불쾌하다', '.']]
{'.': 0, '도': 1, '이다': 2, '좋다': 3, '별로': 4, '다': 5, '이': 6, '너무': 7, '음식': 8, '서비스': 9, '하다': 10, '방문': 11, '위생': 12, '좀': 13, '더': 14, '에': 15, '조금': 16, '정말': 17, '맛있다': 18, '추천': 19, '기대하다': 20, '것': 21, '보단': 22, '가격': 23, '비싸다': 24, '다시': 25, '가다': 26, '싶다': 27, '생각': 28, '안': 29, '드네': 30, '요': 31, '완전': 32, '최고': 33, '!': 34, '재': 35, '의사': 36, '있다': 37, '만족스럽다': 38, '상태

실제 모델에 들어가기 위한 input을 만들기 위해 `Dataset` 클래스를 정의합니다.

In [23]:
class CBOWDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids): 
          self.x.append(token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.y.append(id)

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수, 2 * window_size)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [29]:
class SkipGramDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.y += (token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.x += [id] * 2 * window_size
          
    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

각 모델에 맞는 `Dataset` 객체를 생성합니다.

In [30]:
cbow_set = CBOWDataset(train_tokenized)
skipgram_set = SkipGramDataset(train_tokenized)
print(list(skipgram_set))


100%|██████████| 10/10 [00:00<00:00, 34351.38it/s]

[(tensor(0), tensor(17)), (tensor(0), tensor(18)), (tensor(0), tensor(19)), (tensor(0), tensor(10)), (tensor(19), tensor(18)), (tensor(19), tensor(0)), (tensor(19), tensor(10)), (tensor(19), tensor(0)), (tensor(22), tensor(20)), (tensor(22), tensor(21)), (tensor(22), tensor(4)), (tensor(22), tensor(2)), (tensor(4), tensor(21)), (tensor(4), tensor(22)), (tensor(4), tensor(2)), (tensor(4), tensor(0)), (tensor(23), tensor(5)), (tensor(23), tensor(3)), (tensor(23), tensor(6)), (tensor(23), tensor(7)), (tensor(6), tensor(3)), (tensor(6), tensor(23)), (tensor(6), tensor(7)), (tensor(6), tensor(24)), (tensor(7), tensor(23)), (tensor(7), tensor(6)), (tensor(7), tensor(24)), (tensor(7), tensor(25)), (tensor(24), tensor(6)), (tensor(24), tensor(7)), (tensor(24), tensor(25)), (tensor(24), tensor(26)), (tensor(25), tensor(7)), (tensor(25), tensor(24)), (tensor(25), tensor(26)), (tensor(25), tensor(27)), (tensor(26), tensor(24)), (tensor(26), tensor(25)), (tensor(26), tensor(27)), (tensor(26), tens

### **모델 Class 구현**

차례대로 두 가지 Word2Vec 모델을 구현합니다.  


*   `self.embedding`: `vocab_size` 크기의 one-hot vector를 특정 크기의 `dim` 차원으로 embedding 시키는 layer.
*   `self.linear`: 변환된 embedding vector를 다시 원래 `vocab_size`로 바꾸는 layer.


In [35]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, dim):
    super(CBOW, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x):  # x: (B, 2W)
    embeddings = self.embedding(x)  # (B, 2W, d_w)
    embeddings = torch.sum(embeddings, dim=1)  # (B, d_w) 왜 평균을 안 구하는지
    output = self.linear(embeddings)  # (B, V)
    return output

In [36]:
class SkipGram(nn.Module):
  def __init__(self, vocab_size, dim):
    super(SkipGram, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x): # x: (B)
    embeddings = self.embedding(x)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

두 가지 모델을 생성합니다.

In [37]:
cbow = CBOW(vocab_size=len(w2i), dim=256)
skipgram = SkipGram(vocab_size=len(w2i), dim=256)

### **모델 학습**

다음과 같이 hyperparamter를 세팅하고 `DataLoader` 객체를 만듭니다.

In [38]:
batch_size=4
learning_rate = 5e-4
num_epochs = 5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

cbow_loader = DataLoader(cbow_set, batch_size=batch_size)
skipgram_loader = DataLoader(skipgram_set, batch_size=batch_size)

첫번째로 CBOW 모델 학습입니다.

In [39]:
cbow.train()
cbow = cbow.to(device)
optim = torch.optim.SGD(cbow.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(cbow_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = cbow(x)  # (B, V)
 
    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/16 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 4.361729621887207
Train loss: 5.553689002990723
Train loss: 4.872152328491211
Train loss: 5.099614143371582
Train loss: 4.8152923583984375
Train loss: 5.830530643463135
Train loss: 5.213549613952637
Train loss: 5.005040645599365
Train loss: 4.352278709411621
Train loss: 6.059813499450684
Train loss: 5.066593170166016
Train loss: 5.553886413574219
Train loss: 5.1624579429626465
Train loss: 4.621767520904541
Train loss: 5.176929473876953
Train loss: 4.266761779785156
##################################################
Epoch: 2
Train loss: 4.180353164672852
Train loss: 5.4204912185668945
Train loss: 4.745301723480225
Train loss: 4.981489181518555
Train loss: 4.687221527099609
Train loss: 5.499593734741211


100%|██████████| 16/16 [00:00<00:00, 715.39it/s]

Train loss: 5.008970737457275
Train loss: 4.855337142944336
Train loss: 4.23807954788208
Train loss: 5.839825630187988
Train loss: 4.866974830627441
Train loss: 5.112966537475586
Train loss: 5.026206016540527
Train loss: 4.484927177429199
Train loss: 4.996918201446533
Train loss: 4.132694244384766
##################################################
Epoch: 3
Train loss: 4.0037384033203125
Train loss: 5.2892255783081055
Train loss: 4.619738578796387
Train loss: 4.8648271560668945
Train loss: 4.561341762542725
Train loss: 5.178956985473633
Train loss: 4.8092780113220215
Train loss: 4.709595203399658
Train loss: 4.127050399780273
Train loss: 5.623699188232422
Train loss: 4.673162460327148
Train loss: 4.687160491943359
Train loss: 4.891834259033203
Train loss: 4.351266384124756
Train loss: 4.819890022277832
Train loss: 4.002385139465332
##################################################
Epoch: 4
Train loss: 3.831840991973877
Train loss: 5.1598005294799805
Train loss: 4.49549674987793
Train l

다음으로 Skip-gram 모델 학습입니다.

In [40]:
skipgram.train()
skipgram = skipgram.to(device)
optim = torch.optim.SGD(skipgram.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(skipgram_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = skipgram(x)  # (B, V)

    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/64 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 4.418552398681641
Train loss: 4.998612403869629
Train loss: 4.189716339111328
Train loss: 4.50930118560791
Train loss: 4.3430585861206055
Train loss: 4.459402084350586
Train loss: 4.160971164703369
Train loss: 4.904258728027344
Train loss: 4.015072822570801
Train loss: 4.402087688446045
Train loss: 4.202173709869385
Train loss: 4.707167625427246
Train loss: 3.858921766281128
Train loss: 4.094736099243164
Train loss: 5.167797088623047
Train loss: 4.149286270141602
Train loss: 4.834717750549316
Train loss: 4.4451398849487305
Train loss: 3.949012517929077
Train loss: 4.269545078277588
Train loss: 3.7399449348449707
Train loss: 4.352134704589844
Train loss: 4.5132598876953125
Train loss: 3.43649959564209
Train loss: 3.592538833618164
Train loss: 4.079185485839844
Train loss: 4.0794219970703125
Train loss: 4.3830246925354
Train loss: 4.401734828948975
Train loss: 3.896967887878418
Train loss: 4.18911075592041
Train loss

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 4.010544300079346
Train loss: 4.136783599853516
Train loss: 3.567817211151123
Train loss: 3.9759018421173096
Train loss: 4.376559734344482
Train loss: 3.8455169200897217
Train loss: 4.346393585205078
Train loss: 3.979640483856201
Train loss: 3.969911575317383
Train loss: 4.227710247039795
Train loss: 4.532289505004883
Train loss: 4.401263236999512
Train loss: 4.0854058265686035
Train loss: 4.120846748352051
Train loss: 4.1187872886657715
Train loss: 4.306018829345703
Train loss: 4.573902130126953
Train loss: 4.2010345458984375
Train loss: 4.392168045043945
Train loss: 4.039783000946045
Train loss: 4.695517539978027
##################################################
Epoch: 3
Train loss: 4.345399856567383
Train loss: 4.8899030685424805
Train loss: 4.11378288269043
Train loss: 4.407886505126953
Train loss: 4.272377967834473
Train loss: 4.393229007720947
Train loss: 4.089619159698486
Train loss: 4.842668533325195
Train loss: 3.941926956176758
Train loss: 4.335052490234375
Train

100%|██████████| 64/64 [00:00<00:00, 791.12it/s]

##################################################
Epoch: 5
Train loss: 4.277043342590332
Train loss: 4.781853199005127
Train loss: 4.0390706062316895
Train loss: 4.309793472290039
Train loss: 4.202811241149902
Train loss: 4.328438758850098
Train loss: 4.020605087280273
Train loss: 4.781462669372559
Train loss: 3.869875907897949
Train loss: 4.268455505371094
Train loss: 4.077681541442871
Train loss: 4.600752830505371
Train loss: 3.758148193359375
Train loss: 3.981867790222168
Train loss: 4.9931254386901855
Train loss: 4.039161682128906
Train loss: 4.700235366821289
Train loss: 4.326477527618408
Train loss: 3.8606674671173096
Train loss: 4.143021106719971
Train loss: 3.3485209941864014
Train loss: 3.9307150840759277
Train loss: 4.291937828063965
Train loss: 3.3372950553894043
Train loss: 3.4463725090026855
Train loss: 3.88339900970459
Train loss: 3.9225621223449707
Train loss: 4.255185604095459
Train loss: 4.2587571144104
Train loss: 3.8030920028686523
Train loss: 4.057085990905762
Trai

### **테스트**

학습된 각 모델을 이용하여 test 단어들의 word embedding을 확인합니다.

In [31]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = cbow.embedding(input_id)

  print(f"Word: {word}")
  print(emb.squeeze(0))

NameError: ignored

In [41]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = skipgram.embedding(input_id)

  print(f"Word: {word}")
  print(max(emb.squeeze(0)))

Word: 음식
tensor(2.8897, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 맛
tensor(3.1442, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 서비스
tensor(2.4952, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 위생
tensor(2.8184, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 가격
tensor(2.9439, device='cuda:0', grad_fn=<UnbindBackward>)


In [ ]:
!apt-get install -qq texlive texlive-xetex texlive-latex-extra pandoc
!pip install -qq pypandoc

from google.colab import drive
drive.mount('/content/drive')

!jupyter nbconvert --to PDF '/content/drive/My Drive/Colab Notebooks/1_naive_bayes.ipynb의 사본'